In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd "drive/MyDrive/nlp"

In [ ]:
!pip install -q -U trl transformers accelerate peft datasets bitsandbytes evaluate git+https://github.com/huggingface/huggingface_hub

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [1]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer, BitsAndBytesConfig

model_name = "EMBEDDIA/crosloengual-bert"

In [2]:
from datasets import load_dataset

dataset = load_dataset("lenatr99/Slovene_SuperGLUE_BoolQ")

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

/opt/anaconda3/envs/orange4/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
def preprocess_data(examples):

  passage_inputs = [f"passage : {x} " for x in examples["passage"]]
  question_inputs = [f"question : {x} " for x in examples["question"]]
  inputs = [passage_input + question_input for passage_input, question_input in zip(passage_inputs, question_inputs)]

  model_inputs = tokenizer(inputs, max_length=400, truncation=True)

  model_inputs["labels"] = examples["label"]
  model_inputs["labels"] = [int(label) for label in model_inputs["labels"]]

  return model_inputs

tokenized_dataset = dataset.map(preprocess_data, remove_columns=['idx', 'passage', 'label', 'question'], batched=True)

tokenized_train = tokenized_dataset["train"]
tokenized_val = tokenized_dataset["eval"]

In [5]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [6]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [7]:
id2label = {0: "False", 1: "True"}
label2id = {"False": 0, "True": 1}

model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=2, id2label=id2label, label2id=label2id
)

model.config.use_cache = False

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at EMBEDDIA/crosloengual-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from peft import  get_peft_model, PromptTuningConfig, TaskType, PromptTuningInit

NUM_VIRTUAL_TOKENS = 12

peft_config = PromptTuningConfig(
    peft_type="PROMPT_TUNING",
    task_type=TaskType.SEQ_CLS,
    num_virtual_tokens=NUM_VIRTUAL_TOKENS,
    num_layers=6,
    token_dim=768,
    num_attention_heads=12,
    tokenizer_name_or_path=model_name #The pre-trained model
)

In [9]:
model = get_peft_model(model, peft_config)
print(model.print_trainable_parameters())

trainable params: 10,754 || all params: 124,147,204 || trainable%: 0.0087
None


In [10]:
from transformers import TrainingArguments

new_model_name = "prompt_fine_tuned_boolq_croslo"

training_args = TrainingArguments(
    output_dir=new_model_name,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=50,
    evaluation_strategy='steps',
    max_steps=400,
    use_cpu=False,
    load_best_model_at_end=True
)

In [11]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

max_steps is given, it will override any value given in num_train_epochs


In [12]:
trainer.train()

  0%|          | 0/400 [00:00<?, ?it/s]

{'loss': 0.6715, 'grad_norm': 3.2230186462402344, 'learning_rate': 1.7500000000000002e-05, 'epoch': 4.17}


  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 0.6809973120689392, 'eval_accuracy': 0.6666666666666666, 'eval_f1': 0.6222222222222222, 'eval_runtime': 0.6694, 'eval_samples_per_second': 26.888, 'eval_steps_per_second': 4.481, 'epoch': 4.17}
{'loss': 0.6634, 'grad_norm': 2.8909521102905273, 'learning_rate': 1.5000000000000002e-05, 'epoch': 8.33}


  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 0.6435558795928955, 'eval_accuracy': 0.7777777777777778, 'eval_f1': 0.6805555555555556, 'eval_runtime': 0.3297, 'eval_samples_per_second': 54.589, 'eval_steps_per_second': 9.098, 'epoch': 29.17}
{'loss': 0.6584, 'grad_norm': 2.5921568870544434, 'learning_rate': 0.0, 'epoch': 33.33}


  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 0.6429634094238281, 'eval_accuracy': 0.7777777777777778, 'eval_f1': 0.6805555555555556, 'eval_runtime': 0.5806, 'eval_samples_per_second': 31.0, 'eval_steps_per_second': 5.167, 'epoch': 33.33}
{'train_runtime': 114.228, 'train_samples_per_second': 28.014, 'train_steps_per_second': 3.502, 'train_loss': 0.658894624710083, 'epoch': 33.33}


TrainOutput(global_step=400, training_loss=0.658894624710083, metrics={'train_runtime': 114.228, 'train_samples_per_second': 28.014, 'train_steps_per_second': 3.502, 'total_flos': 390058866032160.0, 'train_loss': 0.658894624710083, 'epoch': 33.333333333333336})

In [13]:
# save model to hub
model_location = "lenatr99" + new_model_name
trainer.push_to_hub(model_location)

/opt/anaconda3/envs/orange4/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/43.3k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/lenatr99/prompt_fine_tuned_boolq_croslo/commit/27d93839f4d47fb89b48beb594f37b46ca59beb9', commit_message='lenatr99prompt_fine_tuned_boolq_croslo', commit_description='', oid='27d93839f4d47fb89b48beb594f37b46ca59beb9', pr_url=None, pr_revision=None, pr_num=None)

In [14]:
trainer.evaluate()

  0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 0.6429634094238281,
 'eval_accuracy': 0.7777777777777778,
 'eval_f1': 0.6805555555555556,
 'eval_runtime': 0.7324,
 'eval_samples_per_second': 24.577,
 'eval_steps_per_second': 4.096,
 'epoch': 33.333333333333336}

In [15]:
# Example
text="passage : Bankovec za 20 evrov – Zaenkrat obstaja le ena celotna serija evrskih bankovcev, vendar pa izhaja nova serija, ki bo podobna sedanji. Evropska centralna banka bo pravočasno naznanila, kdaj bodo bankovci iz prve serije izgubili status zakonitega plačilnega sredstva. question : Ali je bankovec za 20 evrov iz prve serije še vedno zakonito plačilno sredstvo?"

classifier = pipeline("sentiment-analysis", model='tjasad/prompt_fine_tuned_boolq')
classifier(text)

/opt/anaconda3/envs/orange4/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/750 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/490 [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[{'label': 'LABEL_0', 'score': 0.5092487931251526}]